In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import os
from pymongo import MongoClient
from bson.json_util import dumps
import re

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password123"
host = "127.0.0.1"
port = 31080
database = "aac"
collection = "animals"



#db = AnimalShelter(username, password)
shelter = AnimalShelter(username, password)
df = pd.DataFrame.from_records(shelter.read({}))


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


# Encoding the Grazioso Salvare logo
image_filename = 'grazioso_salvare_logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# DEBUG
# print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Lindsey Module 7 CS340'))),
    html.Hr(),
    html.Div(className='buttonRow', 
            style={'display' : 'flex'},
                children=[
                    html.Button(id='submit-button-one', n_clicks=0, children='Cats'),
                    html.Button(id='submit-button-two', n_clicks=0, children='Dogs')
                ]),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable=False,
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr()
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    data = shelter.read({"Type": filter_type})
    df = pd.DataFrame.from_records(data)
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
    return data, columns

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    # Add code for chart of your choice here (e.g. pie chart)
    # return [dcc.Graph(figure=###)]
    pass

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0, 10]),
                    html.H3("Animal Color"),
                    html.P(dff.iloc[0, 5]),
                    html.H3("Animal ID"),
                    html.P(dff.iloc[0, 2]),
                ])
            ])
        ])
    ]

app.run_server(debug=True)

TypeError: __init__() takes 1 positional argument but 3 were given